### **Jogo dos Oito: Implementação de Métodos de Busca**

<p style="font-size: smaller; text-align: right;">João Dias & Rafael Rodrigues</p>

---

### **Objetivo**

Explorar e implementar dois métodos de busca — um cego (*não informado*) e outro informado — para resolver o problema do Jogo dos Oito. Os métodos escolhidos são **Busca em Amplitude (BFS)** e **Algoritmo A***. Este notebook visa comparar a eficiência e o desempenho de ambos os algoritmos, focando na solução do problema e na análise dos resultados.

---

### **Estrutura do Notebook**

1. **Introdução**  
   Contextualização sobre o Jogo dos Oito e os métodos de busca empregados.

2. **Regras do Jogo**  
   Definição do estado inicial, operações válidas, e o estado objetivo.

3. **Implementação da Busca em Amplitude (BFS)**  
   Desenvolvimento passo a passo do método cego.

4. **Implementação do Algoritmo A***  
   Utilização de heurísticas para um método informado mais eficiente.

5. **Visualização dos Movimentos**  
   Demonstração das jogadas que levam do estado inicial ao estado objetivo.

6. **Comparação de Desempenho**  
   Tempo de execução, número de estados explorados e eficiência de cada método.

7. **Conclusão**  
   Reflexões sobre as vantagens e desvantagens de cada abordagem.

---

### **Introdução**

O Jogo dos Oito é um quebra-cabeça clássico que consiste em um tabuleiro de 3x3, com peças numeradas de 1 a 8 e uma posição vazia. O objetivo é reorganizar as peças de um estado inicial para um estado final predeterminado, utilizando movimentos válidos como deslizar uma peça para a posição vazia.

Os métodos de busca empregados para resolver este problema têm aplicações práticas em Inteligência Artificial, especialmente em problemas de caminho mínimo e otimização. Enquanto a **Busca em Amplitude (BFS)** explora todas as possibilidades indiscriminadamente, o **Algoritmo A*** utiliza heurísticas para priorizar caminhos promissores, tornando a busca mais eficiente.

Neste notebook, serão desenvolvidos e comparados ambos os métodos, enfatizando a implementação do zero e a análise dos resultados.

---

## **Implementação**




*<h4>Importação das Bibliotecas*

In [ ]:
# Bibliotecas necessárias
import numpy as np
import heapq
from collections import deque
import time
import matplotlib.pyplot as plt


## **Definindo o estado do Tabuleiro**

*<h4>Criamos a classe PuzzleState para representar o estado do tabuleiro, armazenando o tabuleiro atual, o movimento realizado, o estado anterior e a profundidade atual.*

In [ ]:
class PuzzleState:
    def __init__(self, board, parent=None, move=None, depth=0):
        self.board = board
        self.parent = parent  # Estado anterior
        self.move = move      # Movimento realizado
        self.depth = depth    # Profundidade atual

    # Representação única para estados visitados
    def to_tuple(self):
        return tuple(map(tuple, self.board))

    # Verificar se é o estado final
    def is_goal(self, goal_state):
        return np.array_equal(self.board, goal_state)

    # Gerar novos estados a partir dos movimentos possíveis
    def generate_moves(self):
        if 0 not in self.board:
            raise ValueError("Estado inválido: o tabuleiro não contém um espaço vazio (0).")

        x, y = np.argwhere(self.board == 0)[0]
        directions = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}
        moves = []

        for move, (dx, dy) in directions.items():
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = self.board.copy()
                new_board[x, y], new_board[new_x, new_y] = new_board[new_x, new_y], new_board[x, y]
                moves.append(PuzzleState(new_board, self, move, self.depth + 1))
        return moves

    # Implementando a comparação para o heapq (usando o custo de A* para ordenação)
    def __lt__(self, other):
        return (self.depth + heuristic(self.board, goal_state)) < (other.depth + heuristic(other.board, goal_state))


## **Exibição do Tabuleiro**

*<h4>A função de exibição irá atualizar o tabuleiro de maneira visual a cada movimento. Aqui, estamos utilizando o matplotlib para desenhar o tabuleiro no formato de uma grade.*

In [ ]:
def plot_board(board):
    # Criação de um gráfico
    fig, ax = plt.subplots(figsize=(5, 5))

    # Plota o tabuleiro com fundo branco e bordas escuras
    ax.matshow(np.ones_like(board) * 255, cmap="Blues")  # Cor de fundo branca

    # Adiciona números nas células, e define a cor do texto
    for (i, j), val in np.ndenumerate(board):
        if val != 0:  # Ignora o espaço vazio (0)
            ax.text(j, i, str(val), ha='center', va='center', fontsize=20, color='black')

    # Define limites e remove eixos
    ax.set_xticks(np.arange(-0.5, 3, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 3, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle='-', linewidth=2)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    # Exibe o gráfico
    plt.show()


## **Busca em Amplitude (Breadth-First Search)**

*<h4>A busca em amplitude expande os nós de forma nível a nível, visitando todos os vizinhos antes de ir mais fundo. É um método cego, ou seja, não utiliza informações adicionais como heurísticas.*

In [ ]:
def breadth_first_search(initial_state, goal_state):
    visited = set()
    queue = deque([initial_state])

    while queue:
        state = queue.popleft()

        # Exibir o tabuleiro após cada movimento
        plot_board(state.board)
        
        if state.is_goal(goal_state):
            return state

        visited.add(state.to_tuple())

        for move in state.generate_moves():
            if move.to_tuple() not in visited:
                queue.append(move)
    return None


## **Busca A Star**

*<h4>A busca A estrela é um algoritmo informado que utiliza uma heurística para calcular a distância estimada até o objetivo, melhorando a eficiência da busca.*

In [ ]:
def a_star_search(initial_state, goal_state):
    visited = set()
    priority_queue = []
    heapq.heappush(priority_queue, (0, initial_state))  # (prioridade, estado)

    while priority_queue:
        _, state = heapq.heappop(priority_queue)

        plot_board(state.board)  # Exibe o tabuleiro

        if state.is_goal(goal_state):
            return state

        visited.add(state.to_tuple())

        for move in state.generate_moves():
            if move.to_tuple() not in visited:
                cost = move.depth + heuristic(move.board, goal_state)
                heapq.heappush(priority_queue, (cost, move))
    return None




## **Função Heurística**

In [ ]:
def heuristic(board, goal_state):
    distance = 0
    for i in range(3):
        for j in range(3):
            if board[i][j] != 0:
                goal_pos = np.argwhere(goal_state == board[i][j])[0]
                x, y = goal_pos
                distance += abs(x - i) + abs(y - j)
    return distance


## **Função Principal de Execução**

In [ ]:
# Estado inicial e objetivo
initial_board = np.array([
    [1, 2, 3],
    [4, 0, 6],
    [7, 5, 8]
])

goal_board = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])

initial_state = PuzzleState(initial_board)
goal_state = goal_board

# Testando Busca em Amplitude
start = time.time()
result_bfs = breadth_first_search(initial_state, goal_state)
end = time.time()
print("Busca em Amplitude:")
print(f"Tempo de execução: {end - start:.4f} segundos\n")

# Testando Busca A*
start = time.time()
result_a_star = a_star_search(initial_state, goal_state)
end = time.time()
print("Busca A*: ")
print(f"Tempo de execução: {end - start:.4f} segundos\n")


## **Conclusão**

Neste notebook, comparamos dois algoritmos de busca aplicados ao Jogo dos Oito:

Busca em Amplitude: Um método cego que explora o espaço de estados sem qualquer informação adicional, mais lento para grandes estados.
Busca A*: Um método informado que utiliza uma heurística, mais eficiente em encontrar soluções rápidas.
Ambos os algoritmos foram implementados e testados, com o tempo de execução sendo medido para avaliar a eficiência de cada abordagem.

